<a href="https://colab.research.google.com/github/cs145442/nlp-projects-with-tf2/blob/master/sentiment_classification_with_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Gathering the data

In [ ]:
# add and unzip the dataset here
! ls
! wget http://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip
! unzip stanfordSentimentTreebank.zip

__MACOSX  sample_data  stanfordSentimentTreebank  stanfordSentimentTreebank.zip
--2020-11-21 07:36:01--  http://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip [following]
--2020-11-21 07:36:01--  https://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6372817 (6.1M) [application/zip]
Saving to: ‘stanfordSentimentTreebank.zip.1’

stanfordSentimentTr 100%[===================>]   6.08M  2.36MB/s    in 2.6s    

2020-11-21 07:36:04 (2.36 MB/s) - ‘stanfordSentimentTreebank.zip.1’ saved [6372817/6372817]

Archive:  stanfordSentimentTreebank.zip
replace stanfordSentimentTreeba

In [ ]:
! cat stanfordSentimentTreebank/README.txt

Stanford Sentiment Treebank V1.0

This is the dataset of the paper:

Recursive Deep Models for Semantic Compositionality Over a Sentiment Treebank
Richard Socher, Alex Perelygin, Jean Wu, Jason Chuang, Christopher Manning, Andrew Ng and Christopher Potts
Conference on Empirical Methods in Natural Language Processing (EMNLP 2013)

If you use this dataset in your research, please cite the above paper.

@incollection{SocherEtAl2013:RNTN,
title = {{Parsing With Compositional Vector Grammars}},
author = {Richard Socher and Alex Perelygin and Jean Wu and Jason Chuang and Christopher Manning and Andrew Ng and Christopher Potts},
booktitle = {{EMNLP}},
year = {2013}
}

This file includes:
1. original_rt_snippets.txt contains 10,605 processed snippets from the original pool of Rotten Tomatoes HTML files. Please note that some snippet may contain multiple sentences.

2. dictionary.txt contains all phrases and their IDs, separated by a vertical line |

3. sentiment_labels.txt contains all phrase 

In [ ]:
# take a peek at the dataset format
! echo "----- contents of the treebank -------------------"
! ls stanfordSentimentTreebank
! echo "----- first 5 lines of dictionary.txt ------------"
! tail -n 5 stanfordSentimentTreebank/dictionary.txt
! echo "----- first 5 lines of sentiment_labels.txt ------"
! tail -n 5 stanfordSentimentTreebank/sentiment_labels.txt

----- contents of the treebank -------------------
datasetSentences.txt  dictionary.txt		README.txt	      SOStr.txt
datasetSplit.txt      original_rt_snippets.txt	sentiment_labels.txt  STree.txt
----- first 5 lines of dictionary.txt ------------
zoning ordinances to protect your community from the dullest science fiction|220441
zzzzzzzzz|179256
élan|220442
É|220443
É um passatempo descompromissado|220444
----- first 5 lines of sentiment_labels.txt ------
239227|0.36111
239228|0.38889
239229|0.33333
239230|0.88889
239231|0.5


In [ ]:
# install all the dependencies here
! pip install bert-for-tf2

     |████████████████████████████████| 51kB 4.1MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.7-cp36-none-any.whl size=30539 sha256=9c138f47dc942bf8d0d918456c50ae12b22e5070525c25ea1b30b75f12342795
  Stored in directory: /root/.cache/pip/wheels/e1/f8/e2/b98f79a6b8cc898d8e4102b83acb8a098df7d27500a2bac912
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7304 sha256=65094fb259382c85162a9a57457292f5cd5783ab1c004cacda9cdb94fe60f69a
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19475 sha256=eabae000650f867ba78d265e9ac002572241f7fd270629670dd95d2315f8d6eb
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow


In [ ]:
# import all the dependencies here
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub
import pandas as pd
import bert

import random

In [ ]:
# reading the dataset
dataset_df = pd.read_csv('stanfordSentimentTreebank/dictionary.txt', sep='\n')
dataset_df.head()

,!|0
0,! '|22935
1,! ''|18235
2,! Alas|179257
3,! Brilliant|22936
4,! Brilliant !|40532


In [ ]:
# formatting the dataframe for processing
dataset_df['phrase_text'] = dataset_df['!|0'].apply(lambda x: x.split('|')[0])
dataset_df['phrase_ids'] = dataset_df['!|0'].apply(lambda x: x.split('|')[1])
dataset_df = dataset_df.drop('!|0', axis=1)

In [ ]:
# take a peek at the dataframe
dataset_df.tail()

,phrase_text,phrase_ids
239226,zoning ordinances to protect your community fr...,220441
239227,zzzzzzzzz,179256
239228,élan,220442
239229,É,220443
239230,É um passatempo descompromissado,220444


In [ ]:
# reading the sentiment data
sentiment_df = pd.read_csv('stanfordSentimentTreebank/sentiment_labels.txt', sep='\n')
sentiment_df.head()

,phrase ids|sentiment values
0,0|0.5
1,1|0.5
2,2|0.44444
3,3|0.5
4,4|0.42708


In [ ]:
# formatting the sentiment dataframe for processing
sentiment_df['phrase_ids'] = sentiment_df['phrase ids|sentiment values'].apply(lambda x: x.split('|')[0])
sentiment_df['sentiment_values'] = sentiment_df['phrase ids|sentiment values'].apply(lambda x: x.split('|')[1])
sentiment_df = sentiment_df.drop('phrase ids|sentiment values', axis=1)

In [ ]:
sentiment_df.head()

,phrase_ids,sentiment_values
0,0,0.5
1,1,0.5
2,2,0.44444
3,3,0.5
4,4,0.42708


In [ ]:
# let's merge the phrases and sentiments
dataset_sentiment_df = pd.merge(left=dataset_df, right=sentiment_df, how='inner', on='phrase_ids')
# let's also validate the number of datapoints
print(f"dataset df shape: {dataset_df.shape}")
print(f"sentiment df shape: {sentiment_df.shape}")
print(f"dataset_sentiment df shape: {dataset_sentiment_df.shape}")

dataset df shape: (239231, 2)
sentiment df shape: (239232, 2)
dataset_sentiment df shape: (239231, 3)


*seems good. we missed one datapoint while merging, that's okay for now.*

In [ ]:
def recover_sentiment_class(sentiment_value: float):
  """
  recovering classes from sentiment_values
  [very negative, negative, neutral, positive, very positive]
  [0, 0.2], (0.2, 0.4], (0.4, 0.6], (0.6, 0.8], (0.8, 1.0]
  [1, 2, 3, 4, 5]
  respectively
  :params:
    sentiment_value: floating value of sentiment
  """
  if sentiment_value <= 0.2:
    return 1
  elif sentiment_value <= 0.4:
    return 2
  elif sentiment_value <= 0.6:
    return 3
  elif sentiment_value <= 0.8:
    return 4
  else:
    return 5

In [ ]:
dataset_sentiment_df['sentiment_class'] = dataset_sentiment_df['sentiment_values'].apply(
    lambda x: recover_sentiment_class(float(x)))

In [ ]:
dataset_sentiment_df.tail()

,phrase_text,phrase_ids,sentiment_values,sentiment_class
239226,zoning ordinances to protect your community fr...,220441,0.13889,1
239227,zzzzzzzzz,179256,0.19444,1
239228,élan,220442,0.51389,3
239229,É,220443,0.5,3
239230,É um passatempo descompromissado,220444,0.5,3


## 2. Preprocessing the data

In [ ]:
# let's setup the tokenizer
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)

In [ ]:
# define the vocab and tokenizer from the bert_layer here
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [ ]:
# simple function to encode the sentence
def encode_sentence(sent):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent))

*we're using bert layer for tokenization only!*

In [ ]:
tokenizer.tokenize("don't be so judgemental")

['don', "'", 't', 'be', 'so', 'judgement', '##al']

*masked language model tokenizer, email me for any queries*

In [ ]:
list_of_pharses = list(dataset_sentiment_df['phrase_text'])
encoded_phrases = [encode_sentence(phrase) for phrase in list_of_pharses]

In [ ]:
y = dataset_sentiment_df['sentiment_class']

In [ ]:
list_of_pharses_with_length = [[phrase, y[i], len(phrase)]
                 for i, phrase in enumerate(encoded_phrases)]

In [ ]:
random.shuffle(list_of_pharses_with_length)

*shuffling, something we should always do for better tangling*

In [ ]:
list_of_pharses_with_length.sort(key=lambda x: x[2])

*to handle the dimension for each sequence model, we pad the sequence as per batch size.*

In [ ]:
sorted_phrases_sentiments = [(phrase_with_length[0], phrase_with_length[1]) for phrase_with_length in list_of_pharses_with_length]

In [ ]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_phrases_sentiments, output_types=(tf.int32, tf.int32))

In [ ]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [ ]:
next(iter(batched_dataset))

(<tf.Tensor: shape=(32, 1), dtype=int32, numpy=
 array([[ 3940],
        [ 4091],
        [ 4589],
        [13897],
        [ 7897],
        [11308],
        [ 3544],
        [ 5758],
        [16539],
        [ 9657],
        [15133],
        [13995],
        [ 4550],
        [ 4832],
        [ 5789],
        [ 2742],
        [21443],
        [ 9210],
        [ 4759],
        [ 2811],
        [ 4751],
        [ 3163],
        [11552],
        [12481],
        [16725],
        [ 2215],
        [ 4199],
        [11892],
        [ 2375],
        [ 5699],
        [20403],
        [ 2518]], dtype=int32)>, <tf.Tensor: shape=(32,), dtype=int32, numpy=
 array([3, 3, 3, 3, 2, 3, 3, 3, 3, 5, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        4, 3, 3, 3, 3, 4, 3, 3, 3, 3], dtype=int32)>)

In [141]:
TOTAL_BATCHES = math.ceil(len(list_of_pharses_with_length) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

*keeping the 10% of the batched dataset for evaluation*